In [4]:
import numpy as np
import os
import sys
from sentence_transformers import SentenceTransformer
import importlib.util

sys.path.append('/Users/vijaykatta/Documents/Projects/local_doc_search')
sys.path.append('/Users/vijaykatta/Documents/Projects/local_doc_search/utils')
from utils.file_system_utils import FolderNode, build_tree, print_tree, fetch_parents, embedding_model

In [24]:
# Specify the root directory you want to analyze
root_directory = '/Users/vijaykatta/Documents/Projects/local_doc_search/tests/test_folder'
#root_directory = '/Users/vijaykatta/Documents'
# Build the tree
tree_root = build_tree(root_directory)

# Print the tree structure
#print_tree(tree_root)

# Example usage: Fetch and print parents of a given node
# Assuming you have a reference to a node `some_node`
some_node = tree_root.children[-1]  # Example: Get the first child of root for demonstration
parents = fetch_parents(some_node)
print("\nParents from the given node to the root:")
for parent in reversed(parents):  # Reverse to print from root to the given node
    print(f'Name: {parent.name}, Path: {parent.path}')


Parents from the given node to the root:
Name: test_folder, Path: /Users/vijaykatta/Documents/Projects/local_doc_search/tests/test_folder


In [25]:
def find_closest_node(root, query_vector, max_similarity=-np.inf, closest_node=None):
    """
    Finds the node whose README vector is closest to the query vector in terms of cosine similarity.
    """
    if root.readme_vector is not None:
        similarity = np.dot(query_vector, root.readme_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(root.readme_vector))
        if similarity > max_similarity:  # Changed to check for greater similarity
            max_similarity = similarity
            closest_node = root

    for child in root.children:
        closest_node, max_similarity = find_closest_node(child, query_vector, max_similarity, closest_node)

    return closest_node, max_similarity

def search_and_read_readmes(root, query):
    query_vector = embedding_model.encode(query)
    closest_node, _ = find_closest_node(root, query_vector)

    # Read READMEs from the closest node to the root
    current_node = closest_node
    while current_node is not None:
        if current_node.readme_content:
            print(f"folder_path: {os.path.abspath(current_node.path)}")
        current_node = current_node.parent

# Usage
tree_root = build_tree(root_directory)  # Build the tree
search_and_read_readmes(tree_root, "testing")

folder_path: /Users/vijaykatta/Documents/Projects/local_doc_search/tests/test_folder
